## Test Script for Early Analysis

Dieses Skript dient dazu, sich mit der Datnstruktur von [oeffentlichevergabe.de](https://oeffentlichevergabe.de/ui/de/) vetraut zu machen.

Ziel des Skript ist es:
- Zu überprüfen wie vollständig die Daten sind
- Ob und welche Risikoindikatoren auf Grundlage der Daten gebildet werden können

## 1. Libraries

In [ ]:

import pandas as pd # for prduction change to polas?

#Improve paths later on. Run everything from project root without notebooks
import sys
import os
# Add project root (two levels up) to sys.path
sys.path.append(os.path.abspath(os.path.join('..', '..')))

# Import risk analysisi functions
from utils.analysis.risk_analysis import analyze_risk


In [ ]:
# Test risk analysis function
analyze_risk('test')

testing imports


## 2. Load & Examine Structure of individual xlsx sheets
note: The individual csv files that the main.py scropt doenloads are combined into one xlsx file (stand: 18.12.25). This provides the main entry point for us to investigate the retrieved data

In [17]:
# Load the Excel file
df1 = pd.ExcelFile('../../data/notices_2025-12-06.xlsx')

In [18]:
for sheet_name in df1.sheet_names:
    df = df1.parse(sheet_name)
    print(f"\n=== {sheet_name} ===")
    print(df.columns.tolist())
    print(df.head(2))  # optional but very helpful


=== additionalInformation ===
['noticeIdentifier', 'noticeVersion', 'lotIdentifier', 'suitableForSMEs']
                       noticeIdentifier  noticeVersion lotIdentifier  \
0  18d7ea5a-47a5-49e2-b7c0-676c517d32c6              1      LOT-0001   
1  8e4a0647-5849-4246-912c-a49fccc01fac              1      LOT-0001   

   suitableForSMEs  
0              NaN  
1              0.0  

=== changes ===
['noticeIdentifier', 'noticeVersion', 'contractIdentifier', 'changeNoticeVersionIdentifier', 'changeReasonCode', 'changeReasonDescription']
  noticeIdentifier  noticeVersion contractIdentifier  \
0         24719896              2                NaN   
1         24725244              2                NaN   

  changeNoticeVersionIdentifier changeReasonCode  \
0                             1       update-add   
1                             1       update-add   

           changeReasonDescription  
0  geänderte Daten service.bund.de  
1  geänderte Daten service.bund.de  

=== classification =

In [7]:
# List all sheet names
for sheet_name in df.sheet_names:
    print(sheet_name)

additionalInformation
changes
classification
contract
cvdInformation
duration
lot
notice
noticeResult
organisation
placeOfPerformance
procedure
procedureLotResult
purpose
receivedSubmissions
secondStage
strategicProcurement
submissionTerms
tender


In [13]:
# testing organisation data
organisation = pd.read_excel('../../data/notices_2025-12-06.xlsx',sheet_name='organisation')
organisation.head(2)

,noticeIdentifier,noticeVersion,organisationName,organisationIdentifier,organisationCity,organisationPostCode,organisationCountrySubdivision,organisationCountryCode,organisationInternetAddress,organisationNaturalPerson,organisationRole,buyerProfileURL,buyerLegalType,buyerContractingEntity,winnerSize,winnerOwnerNationality,winnerListed
0,18d7ea5a-47a5-49e2-b7c0-676c517d32c6,1,Landkreis Cuxhaven,Leitweg-ID 03352-0-30,Cuxhaven,27474,DE932,DEU,https://www.landkreis-cuxhaven.de,NaN,buyer,NaN,koerp-oer-kommun,NaN,NaN,NaN,NaN
1,18d7ea5a-47a5-49e2-b7c0-676c517d32c6,1,Vergabekammer Niedersachsen beim Nds. Minister...,t:04131153308,Lüneburg,21339,DE935,DEU,NaN,NaN,reviewer,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
# testing procedureLotResult data
noticeResult = pd.read_excel('../../data/notices_2025-12-06.xlsx',sheet_name='noticeResult')
noticeResult.head(2)

,noticeIdentifier,noticeVersion,noticeValue,noticeValueCurrency,noticeFrameworkValue,noticeFrameworkValueCurrency
0,18d7ea5a-47a5-49e2-b7c0-676c517d32c6,1,571400.0,EUR,NaN,NaN
1,24743392,1,NaN,NaN,NaN,NaN


## Test: SINGLE BIDDING INDICATOR

In [22]:
procedureLotResult_df = pd.read_excel('../../data/notices_2025-12-06.xlsx',sheet_name='procedureLotResult')
procedureLotResult_df.head()

,noticeIdentifier,noticeVersion,lotIdentifier,procedureLotResultNumber,winnerChosen,notAwardedReason,frameworkMaximumValue,frameworkMaximumValueCurrency,frameworkEstimatedValue,frameworkEstimatedValueCurrency,tenderValueLowest,tenderValueLowestCurrency,tenderValueHighest,tenderValueHighestCurrency
0,18d7ea5a-47a5-49e2-b7c0-676c517d32c6,1,LOT-0001,0,selec-w,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,a74e1c14-073f-4e93-8ee1-93d97ebc47bd,1,LOT-0001,0,selec-w,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1797a6ef-de75-4b1a-ae3d-fca153617d25,1,LOT-0001,0,selec-w,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,538d462d-1731-4ee7-bc90-1d3221295ac8,1,LOT-0001,0,selec-w,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,9b9cd0bc-f28e-4362-ace2-8a296e112c6f,1,LOT-0001,0,selec-w,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
receivedSubmissions_df = pd.read_excel('../../data/notices_2025-12-06.xlsx',sheet_name='receivedSubmissions')

receivedSubmissions_df

,noticeIdentifier,noticeVersion,procedureLotResultNumber,receivedSubmissionsCount,receivedSubmissionsType
0,18d7ea5a-47a5-49e2-b7c0-676c517d32c6,1,0,3,t-esubm
1,a74e1c14-073f-4e93-8ee1-93d97ebc47bd,1,0,5,tenders
2,1797a6ef-de75-4b1a-ae3d-fca153617d25,1,0,3,tenders
3,538d462d-1731-4ee7-bc90-1d3221295ac8,1,0,13,part-req
4,9b9cd0bc-f28e-4362-ace2-8a296e112c6f,1,0,4,t-esubm
...,...,...,...,...,...
165,a4014900-78bb-46c1-9dc4-6f47bcc021f3,1,1,3,tenders
166,a4014900-78bb-46c1-9dc4-6f47bcc021f3,1,1,3,t-esubm
167,a4014900-78bb-46c1-9dc4-6f47bcc021f3,1,1,2,t-sme
168,a4014900-78bb-46c1-9dc4-6f47bcc021f3,1,1,0,t-oth-eea


In [ ]:
pd.merge(procedureLotResult_df, receivedSubmissions_df, on=['noticeIdentifier','noticeVersion','procedureLotResultNumber'], how='inner')